# Focus Alert — Log Analysis Notebook

This notebook loads CSV logs, constructs frame/event timelines, and computes basic evaluation metrics (sensitivity/specificity/F1/AUC) and detection delay.

Ground-truth options:
- Use distractor_start/end events as proxy for low-attention intervals.
- Or replace with task performance onset labels if available.


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_fscore_support
pd.options.display.max_columns = 200
sns.set_context('talk')

# Configure your log files here
LOG_PATHS = [
    # Example: r'C:/Users/Mitsuki_M/CascadeProjects/focus-alert/logs/P01_train_1.csv',
]
assert len(LOG_PATHS) > 0, 'Please set LOG_PATHS to your CSV log files.'


In [ ]:
def load_logs(paths):
    frames = []
    events = []
    for p in paths:
        df = pd.read_csv(p)
        df['source'] = str(p)
        # split
        df_meta = df[df['row_type']=='meta'].copy()
        df_evt = df[df['row_type']=='event'].copy()
        df_fr = df[df['row_type']=='frame'].copy()
        # forward-fill meta
        meta = {}
        if len(df_meta):
            last = df_meta.iloc[-1]
            meta = {k: last.get(k) for k in ['session','participant','task','phase']}
        for k,v in meta.items():
            df_evt[k] = v
            df_fr[k] = v
        frames.append(df_fr)
        events.append(df_evt)
    fr = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    ev = pd.concat(events, ignore_index=True) if events else pd.DataFrame()
    return fr, ev

frames, events = load_logs(LOG_PATHS)
frames.sort_values('ts', inplace=True)
events.sort_values('ts', inplace=True)
print(frames.head())
print(events.head())


In [ ]:
# Build distractor_active timeline from events
frames['distractor_active'] = False
active = False
idx = 0
ev_list = events.to_dict('records')
for i, row in frames.iterrows():
    ts = row['ts']
    while idx < len(ev_list) and ev_list[idx]['ts'] <= ts:
        e = ev_list[idx]
        if e.get('event') == 'distractor_start':
            active = True
        elif e.get('event') == 'distractor_end':
            active = False
        idx += 1
    frames.at[i, 'distractor_active'] = active

# Basic derived columns
frames['alert_bool'] = frames['alert'].astype('int').astype('bool')
frames['risk'] = pd.to_numeric(frames['risk'], errors='coerce').fillna(0.0)
frames['block_id'] = frames['block_id'].fillna(-1).astype(int)

frames.head()


In [ ]:
# Quick time-series plot for one session/block
sess = frames['session'].iloc[0] if 'session' in frames and len(frames) else None
dfp = frames if sess is None else frames[frames['session']==sess]
fig, ax = plt.subplots(3,1, figsize=(14,8), sharex=True)
ax[0].plot(dfp['ts'], dfp['risk'], label='risk')
ax[0].scatter(dfp.loc[dfp['alert_bool'],'ts'], dfp.loc[dfp['alert_bool'],'risk'], color='r', s=10, label='alert')
ax[0].legend(); ax[0].set_ylabel('risk')
ax[1].plot(dfp['ts'], dfp['ear'], label='EAR'); ax[1].plot(dfp['ts'], dfp['ear_base'], label='EAR base');
ax[1].legend(); ax[1].set_ylabel('EAR')
ax[2].plot(dfp['ts'], dfp['gaze'], label='gaze'); ax[2].axhline(dfp.get('gaze_thr',0.35).median() if 'gaze_thr' in dfp else 0.35, color='gray', ls='--')
# shade distractor
if 'distractor_active' in dfp:
    y1,y2 = ax[0].get_ylim()
    for _,r in dfp[dfp['distractor_active']].iterrows():
        ax[0].axvspan(r['ts'], r['ts'], color='orange', alpha=0.1)
plt.xlabel('ts')
plt.tight_layout()
plt.show()


In [ ]:
# Classification metrics using distractor_active as proxy GT
gt = frames['distractor_active'].astype(int)
pred_alert = frames['alert_bool'].astype(int)
print('Confusion matrix (Alert vs Distractor):')
print(confusion_matrix(gt, pred_alert))
prec, rec, f1, _ = precision_recall_fscore_support(gt, pred_alert, average='binary', zero_division=0)
print(f'Precision={prec:.3f} Recall={rec:.3f} F1={f1:.3f}')
# AUC using risk as score
try:
    auc = roc_auc_score(gt, frames['risk'])
    print(f'ROC-AUC (risk vs distractor) = {auc:.3f}')
except Exception as e:
    print('AUC calc skipped:', e)


In [ ]:
# Detection delay: time from distractor_start to first alert within a window
starts = events[events['event']=='distractor_start'][['ts','block_id','session']].copy() if len(events) else pd.DataFrame(columns=['ts'])
delays = []
win = 30.0  # seconds window
for _, s in starts.iterrows():
    mask = (frames['ts']>=s['ts']) & (frames['ts']<=s['ts']+win)
    sub = frames[mask]
    a = sub[sub['alert_bool']]
    if len(a):
        delay = a['ts'].iloc[0] - s['ts']
        delays.append(delay)
    else:
        delays.append(np.nan)

if delays:
    s = pd.Series(delays)
    print('Detection delay seconds: median=', s.median(), ' N=', s.notna().sum())
else:
    print('No distractor_start events found.')
